# Susceptible, Infected, Recovered: Influenza in a Boarding School

In 1978, anonymous authors sent a note to the British Medical Journal
reporting an influenza outbreak in a boarding school in the north of
England (@bmj-influenza). The chart below shows the solution of the
SIR (Susceptible, Infected, Record) model with parameters which give
roughly the results observed in the school.

We can model this using the [textbook
model](https://en.wikipedia.org/wiki/Compartmental_models_in_epidemiology) which
is more or less the one that @1927RSPSA.115 published almost 100 years
ago and which treats the number of infected individuals as continuous.

The Susceptible / Infected / Recovered (SIR) model has three
parameters: one describing how infectious the pathogen is ($\beta$), one
describing how much contact a host has with other hosts ($c$) and one
describing how quickly a host recovers ($\gamma$).

$$
\begin{aligned}
\frac{d S}{d t} &=-c \beta S \frac{I}{N} \\
\frac{d I}{d t} &=c \beta S \frac{I}{N}-\gamma I \\
\frac{d R}{d t} &=\gamma I
\end{aligned}
$$

The infectivity rate and the contact rate are always used as $c\beta$
and are thus non-identifiable so we can replace this product with a
single parameter $\alpha = c\beta$).

$$
\begin{aligned}
\frac{d S}{d t} &=-\alpha S \frac{I}{N} \\
\frac{d I}{d t} &=\alpha S \frac{I}{N}-\gamma I \\
\frac{d R}{d t} &=\gamma I
\end{aligned}
$$

In [49]:
{-# LANGUAGE ViewPatterns        #-}
{-# LANGUAGE NumDecimals         #-}
{-# LANGUAGE OverloadedLists     #-}
{-# LANGUAGE FlexibleContexts    #-}
{-# LANGUAGE RecordWildCards     #-}

import           Numeric.Sundials
import           Numeric.LinearAlgebra
import qualified Data.Vector.Storable as VS
import           Control.Monad.Bayes.Class
import qualified Data.Vector as V
import           Control.Exception
import           Control.Monad.IO.Class
import           Katip
import           Katip.Monadic ()
import           System.IO

:l Plotting.hs

In [50]:
data SirState = SirState {
    sirStateS :: Double
  , sirStateI :: Double
  , sirStateR :: Double
  } deriving (Eq, Show)

data SirParams = SirParams {
    sirParamsBeta  :: Double
  , sirParamsC     :: Double
  , sirParamsGamma :: Double
  } deriving (Eq, Show)

data SirParams' = SirParams' {
    sirParamsR0    :: Double
  , sirParamsKappa :: Double
  } deriving (Eq, Show)

data Sir = Sir {
    sirS     :: SirState
  , sirP     :: SirParams
  } deriving (Eq, Show)

data SirReparam = SirReparam {
    sirS'     :: SirState
  , sirP'     :: SirParams'
  } deriving (Eq, Show)


In [51]:
:i SirParams

type SirParams :: *
data SirParams = SirParams {sirParamsBeta :: Double, sirParamsC :: Double, sirParamsGamma :: Double}
  	-- Defined at <interactive>:6:1
instance [safe] Show SirParams -- Defined at <interactive>:10:19
instance [safe] Eq SirParams -- Defined at <interactive>:10:15

In [52]:
:i SunMatrix

type SunMatrix :: *
data SunMatrix = SunMatrix {Numeric.Sundials.rows :: SunIndexType, Numeric.Sundials.cols :: SunIndexType, vals :: Vector Foreign.C.Types.CDouble}
  	-- Defined in ‘hmatrix-sundials-0.20.0.0:Numeric.Sundials.Foreign’
instance VS.Storable SunMatrix -- Defined in ‘hmatrix-sundials-0.20.0.0:Numeric.Sundials.Foreign’

As in most languages, it's easy enough to define the actual ODE
problem itself and then run a solver to return the results and then
plot them. Here we are using a 4-th order implicit method from the
[SUNDIALS ODE solver
package](https://sundials.readthedocs.io/en/latest/arkode/Butcher_link.html#sdirk-5-3-4)
but almost any solver would have worked for the set of equations we
using as our example.

I have hidden the details which can made be visible if the reader is
interested.

In [53]:
defaultOpts :: OdeMethod -> ODEOpts
defaultOpts method = ODEOpts
  { maxNumSteps = 1e5
  , minStep     = 1.0e-14
  , fixedStep   = 0
  , maxFail     = 10
  , odeMethod   = method
  , initStep    = Nothing
  , jacobianRepr = DenseJacobian
  }

emptyOdeProblem :: OdeProblem
emptyOdeProblem = OdeProblem
      { odeRhs = error "emptyOdeProblem: no odeRhs provided"
      , odeJacobian = Nothing
      , odeInitCond = error "emptyOdeProblem: no odeInitCond provided"
      , odeEventDirections = V.empty
      , odeEventConditions = eventConditionsPure V.empty
      , odeTimeBasedEvents = TimeEventSpec $ return $ 1.0 / 0.0
      , odeEventHandler = nilEventHandler
      , odeMaxEvents = 100
      , odeSolTimes = error "emptyOdeProblem: no odeSolTimes provided"
      , odeTolerances = defaultTolerances
      }

nilEventHandler :: EventHandler
nilEventHandler _ _ _ = throwIO $ ErrorCall "nilEventHandler"

defaultTolerances :: Tolerances
defaultTolerances = Tolerances
  { absTolerances = Left 1.0e-6
  , relTolerance = 1.0e-10
  }

In [54]:
:i OdeRhs

type OdeRhs :: *
data OdeRhs = OdeRhsHaskell (Foreign.C.Types.CDouble -> Vector Foreign.C.Types.CDouble -> IO (Vector Foreign.C.Types.CDouble)) | OdeRhsC (GHC.Ptr.FunPtr OdeRhsCType) (GHC.Ptr.Ptr UserData)
  	-- Defined in ‘hmatrix-sundials-0.20.0.0:Numeric.Sundials.Common’

In [55]:
sir :: Vector Double -> Sir -> OdeProblem
sir ts ps = emptyOdeProblem
  { odeRhs = odeRhsPure f
  , odeJacobian = Nothing
  , odeInitCond = [initS, initI, initR]
  , odeEventHandler = nilEventHandler
  , odeMaxEvents = 0
  , odeSolTimes = ts
  , odeTolerances = defaultTolerances
  }
  where
    f _ (VS.toList -> [s, i, r]) =
      let n = s + i + r in
        [ -beta * c * i / n * s
        , beta * c * i / n * s - gamma * i
        , gamma * i
        ]
    f _ _ = error "Incorrect number of parameters"

    beta  = realToFrac (sirParamsBeta  $ sirP ps)
    c     = realToFrac (sirParamsC     $ sirP ps)
    gamma = realToFrac (sirParamsGamma $ sirP ps)
    initS = realToFrac (sirStateS $ sirS ps)
    initI = realToFrac (sirStateI $ sirS ps)
    initR = realToFrac (sirStateR $ sirS ps)

In [56]:
us :: [Double]
us = map fromIntegral [1 .. length actuals]

actuals :: [Double]
actuals = [1, 3, 8, 28, 76, 222, 293, 257, 237, 192, 126, 70, 28, 12, 5]

In [57]:
solK :: (MonadIO m, Katip m) =>
        (a -> b -> OdeProblem) -> b -> a -> m (Matrix Double)
solK s ps ts = do
  x <- solve (defaultOpts $ ARKMethod SDIRK_5_3_4) (s ts ps)
  case x of
    Left e  -> error $ show e
    Right y -> return (solutionMatrix y)

testSolK :: (MonadIO m, KatipContext m) => m [Double]
testSolK = do
  m <- solK sir (Sir (SirState 762 1 0) (SirParams 0.2 10.0 0.5)) (vector us)
  let n = tr m
  return $ toList (n!1)

In [58]:
myBracketFormat :: LogItem a => ItemFormatter a
myBracketFormat _withColor _verb Item {..} =
    unLogStr _itemMessage

In [59]:
main :: IO ()
main = do
  handleScribe <- mkHandleScribeWithFormatter myBracketFormat ColorIfTerminal stderr (permitItem DebugS) V2
  logEnv <- registerScribe "stderr" handleScribe defaultScribeSettings =<< initLogEnv "test" "devel"
  r <- runKatipContextT logEnv (mempty :: LogContexts) mempty testSolK
  print r
  return ()

In [60]:
main

[1.0,4.449470258029982,19.260613955067594,74.56001407651216,203.53132707275714,303.9377394431499,280.8355064841971,208.44313889488538,141.93376055400773,93.16867111576789,60.098401961236064,38.41317069995869,24.42723523310017,15.487105777221927,9.80136250623264]

Definition: Let $\mathcal{C}$ be a category. A monad on $\mathcal{C}$ is a triple

$$
\left(C \stackrel{T}{\longrightarrow} C, T^{2} \stackrel{\mu}{\Longrightarrow} T, I \stackrel{\eta}{\Longrightarrow} T\right)
$$

obeying the standard associativity and identity properties.

${\mathcal{C}}_0$ measurable spaces ${\mathcal{C}}_1$ measurable functions on the former

Objects of ${\mathcal{C}}$ and morphisms

$$
P : (X, \Sigma_X) \longrightarrow (PX, \Sigma_{PX})
$$

$$
\begin{array}{ll}
P X \stackrel{ev_{E}}{\longrightarrow}[0,1] & \text { are measurable } \\
\mu \longmapsto \mu(E) & \forall E \in \sum_{X}
\end{array}
$$

$\Sigma_{PX}$ is the smallest $\sigma$-algebra that makes the evaluation map $ev_E$ measurable.

$$
\begin{aligned}
\left(X, \Sigma_{X}\right) \stackrel{f}{\longrightarrow}\left(Y, \Sigma_{r}\right) & \\
P X \stackrel{P f}{\longrightarrow} P Y \quad(P f)(\mu)=f_{*} \mu \\
\left(f_{* \mu}\right)(A)=\mu\left(f^{-1}(A)\right)
\end{aligned}
$$

Monad on measurable spaces aka the Giry Monad

$$
\left(C \stackrel{T}{\longrightarrow} C, T^{2} \stackrel{\mu}{\Longrightarrow} T, I \stackrel{\eta}{\Longrightarrow} T\right)
$$

The Kleisli categany associated with $(T, \mu, \eta)$ has objets the same as $C$

while a morphism from $a$ to $b$ denoted $a \stackrel{f}\rightsquigarrow b$, is defined to be a morphism $a \stackrel{f}{\longrightarrow} T b$.

$$
T_{c} \stackrel{B_{c}}{\longleftarrow} T_{c}^{2} \stackrel{T_{g}}{\longleftarrow} T_{\hookleftarrow} \stackrel{\&}{\longleftarrow}
$$

$C \stackrel{1_C}\rightsquigarrow C$ is $C \stackrel{\eta_C}\rightarrow TC$ 

${\mathcal{C}} \longrightarrow \mathrm{Kl}(T)$

Giry monad has the Kleisli category of measurable spaces and Markov kernels

Example: let $X = \{0, 1, \ldots, n - 1\}$

$PX = \{(p_0, p_1, \ldots, p_{n-1}) \in \mathbb{R}^n \mid p_j \ge 0 \forall j \& \sum_j p_j =1 \}$ in other words a $n - 1$-simplex.

Recall the $\sigma$-algebra on the space $PX$ is defined as the one which makes the evaluation maps measurable.